In [3]:
!pip install -U spacy



In [4]:
!pip install -U requests==2.27.1


In [8]:
!pip uninstall inflect

Found existing installation: inflect 6.0.4
Uninstalling inflect-6.0.4:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/inflect-6.0.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/inflect/*
Proceed (Y/n)? y
  Successfully uninstalled inflect-6.0.4


In [9]:
!pip install -U pydantic>=1.9.1


In [10]:
!pip install allennlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 123.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 31.9 MB/s eta 0:00:00
  Preparing metad

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch

In [12]:
# Load the dataset into a pandas DataFrame
df = pd.read_csv('/content/Restaurant_Reviews.csv')  # Assuming 'reviews.csv' contains 'text' and 'liked' columns

In [13]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Review'], df['Liked'], test_size=0.2, random_state=42)

In [14]:
# Initialize ELMo
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
elmo = Elmo(options_file, weight_file, num_output_representations=1).to(device)

# Create ELMo embeddings
def get_elmo_embeddings(texts):
    character_ids = batch_to_ids(texts).to(device)
    embeddings = elmo(character_ids)['elmo_representations'][0].detach().cpu().numpy()
    return embeddings


Output()

Output()

In [22]:
batch_size = 32
num_batches = len(X_train) // batch_size

X_train_features = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    batch = X_train[start_idx:end_idx]
    batch_elmo = get_elmo_embeddings(batch)
    batch_features = batch_elmo.reshape(batch_elmo.shape[0], -1)

    X_train_features.append(batch_features)

# Process the remaining samples
remaining_samples = X_train[end_idx:]
if len(remaining_samples) > 0:
    remaining_elmo = get_elmo_embeddings(remaining_samples)
    remaining_features = remaining_elmo.reshape(remaining_elmo.shape[0], -1)
    X_train_features.append(remaining_features)

# Find the maximum size along axis 1
max_size = max([f.shape[1] for f in X_train_features])

# Zero-pad arrays to have consistent dimensions
X_train_features_padded = []
for features in X_train_features:
    pad_size = max_size - features.shape[1]
    padded_features = np.pad(features, ((0, 0), (0, pad_size)), mode='constant')
    X_train_features_padded.append(padded_features)

X_train_features = np.concatenate(X_train_features_padded, axis=0)


In [23]:
# Process the test data
X_test_elmo = get_elmo_embeddings(X_test)
X_test_features = X_test_elmo.reshape(X_test_elmo.shape[0], -1)

In [25]:
print("Shape of X_train_features:", X_train_features.shape)
print("Shape of X_test_features:", X_test_features.shape)


Shape of X_train_features: (800, 150528)
Shape of X_test_features: (200, 152576)


In [ ]:
# Print the shape of the original ELMo embeddings for training data batches
for batch in X_train_features:
    for emb in batch:
        print(emb.shape)


Streaming output truncated to the last 5000 lines.
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
(

In [31]:
# Print the shape of the original ELMo embeddings for training data
for emb in X_train_elmo:
    print(emb.shape)

# Print the shape of the original ELMo embeddings for test data
for emb in X_test_elmo:
    print(emb.shape)


NameError: ignored

In [30]:
# Find the maximum size along axis 1 for training features
max_size_train = max([f.shape[1] for f in X_train_features])

# Print the shapes of the original training features
print("Original training feature shapes:")
for f in X_train_features:
    print(f.shape)

# Pad training arrays to have consistent dimensions
X_train_features_padded = np.concatenate([np.pad(f, ((0, 0), (0, max_size_train - f.shape[1])), mode='constant') if f.shape[1] < max_size_train else f for f in X_train_features])

# Find the maximum size along axis 1 for test features
max_size_test = max([f.shape[1] for f in X_test_features])

# Print the shapes of the original test features
print("Original test feature shapes:")
for f in X_test_features:
    print(f.shape)

# Pad test arrays to have consistent dimensions
X_test_features_padded = np.concatenate([np.pad(f, ((0, 0), (0, max_size_test - f.shape[1])), mode='constant') if f.shape[1] < max_size_test else f for f in X_test_features])

# Print the shapes of the padded training features
print("Padded training feature shape:", X_train_features_padded.shape)

# Print the shapes of the padded test features
print("Padded test feature shape:", X_test_features_padded.shape)


IndexError: ignored

In [ ]:

# Calculate evaluation metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
